# Borrower Reliability Study -- The Analytic Way

The customer is the credit department of the bank. It is necessary to find out whether the **marital status** and the **number of children** of the client affect the fact of repaying the loan on time. Input data from the bank - statistics on the solvency of customers.
The results of the study will be taken into account when building a  **credit scoring model**  a special system that assesses the ability of a potential borrower to return a loan to a bank.

## Step 1 : Open the table and examine the general information about the data

In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
clients = pd.read_csv('data.csv')

#### An overview of all data

In [4]:
clients.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Data Description
<pre>
- <strong>children</strong>             : number of children in the family
- <strong>days_employed</strong>        : total work experience in days
- <strong>dob_years</strong>            : client's age in years
- <strong>education</strong>            : the level of education of the client
- <strong>education_id</strong>         : education level identifier
- <strong>family_status</strong>        : marital status
- <strong>family_status_id</strong>     : marital status identifier
- <strong>gender</strong>               : gender of the client
- <strong>income_type</strong>          : type of employment
- <strong>debt</strong>                 : whether he had loan repayment arrears
- <strong>total_income</strong>         : monthly income
- <strong>purpose</strong>              : the purpose of obtaining a loan
</pre>

In [5]:
print(clients.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


In [6]:
## describe data
clients.describe().loc[['count','min','max','mean']]

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.00000,21525.000000,21525.000000,21525.000000,1.935100e+04
min,-1.000000,-18388.949901,0.00000,0.000000,0.000000,0.000000,2.066726e+04
max,20.000000,401755.400475,75.00000,4.000000,4.000000,1.000000,2.265604e+06
mean,0.538908,63046.497661,43.29338,0.817236,0.972544,0.080883,1.674223e+05


## General Information about the data set
> 1. The dataset contain **21525** two-dimensional data array of clients from the bank. <br>

> 2. Each client is described by  12 columns or features<br>
> five of them are quantitative and other are categorital or binary.
<br>

> 3. children contain minimum value (-1)  and greater value (20). <br>
  &nbsp; &nbsp; &nbsp;- This is probabaly is human mistake. Maybe 1 in place of -1 and 2 in place or 20 .<br>
  &nbsp; &nbsp; &nbsp;- Hire collegue to confirm before correction or deletion<br>

> 4. days_employed column contain missing value about (100 - (19351/21525)*100) == 10% .<br> 
>  &nbsp; &nbsp; &nbsp;- This is not a small value.It is raison to check if  <br>
>   &nbsp; &nbsp; &nbsp;- the missing value in days_employed feature  induce missing  value in t	total_income.
> (The client don't have a job).
>  &nbsp; &nbsp; &nbsp;- recall that we don' t need this columns for our work<br>

> 5. days_employed contain also negative value.<br>
>  &nbsp; &nbsp; &nbsp;-The negatives values can be replace by  the absolute value .<br>
>   &nbsp; &nbsp; &nbsp;-It can be a human or technical error . ask collegue before correction.

> 6. 0 values in the dob_years columns  . Mistake is possible here<br>
>   &nbsp; &nbsp; &nbsp;-It can be a human or technical error . ask collegue before correction.

> 7. The education feature have Upper Case and lower case string<br>
>   &nbsp; &nbsp; &nbsp;- We should standardize.

> ### Conclusion Step 1
> Checklists and process docs should be available to all employees and serve as a way to double-check they’re following proper procedure and limit data entry to a certain range rejecting no allowed value .

## Step2:  Data preprocessing
#### Step2-1 : Dealing with no correct values 

In [7]:
clients

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [8]:
#index where children have negatives values
children_negative_values = np.where(clients['children'] < 0)[0]
print("|Negatives values in 'children feature'| = {len} \n".format(len = clients[ clients['children'] < 0 ].count()['children']))
print("|Values in 'children' feature >= 20| = {len} \n".format(len = clients[ clients['children'] >= 20 ].count()['children']))
print("|Negatives values in 'days_employed' feature| = {len} \n".format(len = clients[ clients['days_employed'] < 0 ].count()['days_employed']))
print("|0 values in 'dob_years' (clients age ) feature| = {len} \n".format(len = clients[ clients['dob_years']  == 0 ].count()['dob_years']))


|Negatives values in 'children feature'| = 47 

|Values in 'children' feature >= 20| = 76 

|Negatives values in 'days_employed' feature| = 15906 

|0 values in 'dob_years' (clients age ) feature| = 101 




&nbsp; &nbsp; &nbsp;   **"Negatives values in 'children feature'| = 47"**.<br>
&nbsp; &nbsp; &nbsp;   **"|Values in 'children' feature >= 20| = 47"**.<br>
&nbsp; &nbsp; &nbsp; **~74%** in **'days_employed' feature are negative** . (You should take the absolute value).<br>
&nbsp; &nbsp; &nbsp; **"|0 values in 'dob_years' (clients age ) feature| = 101**."<br>

Most of code above is for training purpose .
Since it is necessary to find out whether the **marital status** and **the number of children** of the client affect the fact of repaying the loan on time we can delete the 'days_employed' feature .
   

In [9]:
# display rows  where number of chidren are negative 
# clients.iloc[children_negative_values].head()

In [10]:
from IPython.core.display import HTML

In [11]:
# PROCESSING NO CORRECT VALUES

# drop days employed feature
new_clients = clients.drop("days_employed", axis = 1)

# deleting row  where children feature is negative 
new_clients = new_clients.drop(new_clients[new_clients['children'] < 0].index)
# new_clients[new_clients['children'] < 0]
new_clients

# replace 20 by 2 in children feature 
new_clients['children'] = new_clients['children'].replace(20,2)
# print("|Values in 'children' feature >= 20| = {len} \n".format(len = new_clients[ new_clients['children'] >= 20 ].count()['children']))
# check for null values in children
# print(np.where(new_clients['children'].isnull().count()))


# print(new_clients['dob_years'].describe())
new_clients['dob_years'] = new_clients['dob_years'].replace(0,43)
# print("|0 values in 'dob_years' (clients age ) feature| = {len} \n".format(len = new_clients[ new_clients['dob_years']  == 0 ].count()['dob_years']))
# print(np.where(new_clients['dob_years'].isnull().count()))

# to lower all value in education

new_clients['education'] = new_clients['education'].str.lower()
# print(np.where(new_clients['education'].isnull().count()))


new_clients['total_income'].mean()
new_clients['total_income'] = new_clients['total_income'].fillna(167422)
# print(np.where(new_clients['total_income'].isnull().count()))


In [12]:
#CHANGE DATA TYPE
new_clients['total_income'] = new_clients['total_income'].astype(int)


In [13]:
# REMOVE DUPLICATE VALUE 
new_clients =  new_clients.drop_duplicates()
print(new_clients.duplicated().sum())


0


In [14]:
# LEMMATIZATION
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
new_clients['purpose'].value_counts()


свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           764
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
операции с жильем                         651
покупка жилья для сдачи                   650
операции с коммерческой недвижимостью     649
жилье                                     645
покупка жилья                             643
покупка жилья для семьи                   638
строительство собственной недвижимости    634
недвижимость                              632
операции со своей недвижимостью           624
строительство жилой недвижимости          621
строительство недвижимости                619
покупка своего жилья                      619
покупка недвижимости                      618
ремонт жилью                              606
покупка жилой недвижимости                604
на покупку своего автомобиля              504
заняться высшим образованием      

we can divide our text in five group  : **'свадьба','недвижимость','жилье', 'автомобиль', 'бразование'**



In [15]:
purpose_category = ['свадьба','недвижимость','жилье', 'автомобиль', 'бразование']

In [16]:
# LEMMATIZATION

def lemmatizer(element):
    lemma = m.lemmatize(''.join(element))
    for word in purpose_category:
        if word in str(lemma):
             return word
    return lemma


new_clients['purpose_lemma'] = new_clients['purpose'].apply(lemmatizer)
new_clients[['purpose','purpose_lemma']]

,purpose,purpose_lemma
0,покупка жилья,жилье
1,приобретение автомобиля,автомобиль
2,покупка жилья,жилье
3,дополнительное образование,бразование
4,сыграть свадьбу,свадьба
...,...,...
21520,операции с жильем,жилье
21521,сделка с автомобилем,автомобиль
21522,недвижимость,недвижимость
21523,на покупку своего автомобиля,автомобиль


In [17]:
# Count  unique value of lemma to better categorize
new_clients['purpose_lemma'].value_counts()

недвижимость    6335
жилье           4452
автомобиль      4295
бразование      4003
свадьба         2322
Name: purpose_lemma, dtype: int64

In [18]:
new_clients.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21407 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21407 non-null  int64 
 1   dob_years         21407 non-null  int64 
 2   education         21407 non-null  object
 3   education_id      21407 non-null  int64 
 4   family_status     21407 non-null  object
 5   family_status_id  21407 non-null  int64 
 6   gender            21407 non-null  object
 7   income_type       21407 non-null  object
 8   debt              21407 non-null  int64 
 9   total_income      21407 non-null  int64 
 10  purpose           21407 non-null  object
 11  purpose_lemma     21407 non-null  object
dtypes: int64(6), object(6)
memory usage: 2.1+ MB


In [19]:
new_clients.head(15)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,бразование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,бразование
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье


In [20]:
new_clients['children'].value_counts()

0    14091
1     4808
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

**'children'** feature can be divided in 2 groups: <br>
+ 0 and 1 : few children <br>
+ 3 , 4 , 5 : many children <br>

In [21]:
def children(number_children):
    if (number_children <= 1):
        return "few children"
    return "many children"

new_clients['children_grouped'] = new_clients['children'].apply(children)
new_clients

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,children_grouped
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,few children
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,few children
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,few children
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,бразование,many children
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,few children
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,жилье,few children
21521,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,few children
21522,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,few children
21523,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,many children


In [22]:
new_clients['dob_years'].describe()[4:7]

25%    33.0
50%    43.0
75%    53.0
Name: dob_years, dtype: float64

In [23]:
def age(dobt_years):
    if (dobt_years <= 33):
        return "young"
    if(dobt_years <=43):
        return "mature"
    return "old"

In [24]:
new_clients['age'] = new_clients['dob_years'].apply(age)
new_clients

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,children_grouped,age
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,few children,mature
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,few children,mature
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,few children,young
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,бразование,many children,young
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,few children,old
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,жилье,few children,mature
21521,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,few children,old
21522,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,few children,mature
21523,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,many children,mature


In [25]:
stats = new_clients['total_income'].describe()[4:7]
stats

25%    107555.0
50%    156069.0
75%    195854.0
Name: total_income, dtype: float64

In [26]:
def income(total_income):
    if(total_income <= stats[0]):
        return 1 
    if(total_income <= stats[1]):
        return 2 
    if(total_income <= stats[2]):
        return 3
    return 4 
        

In [27]:
new_clients['income_indice'] = new_clients['total_income'].apply(income)
new_clients['income_indice'] = new_clients['income_indice'].astype(int)
new_clients

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,children_grouped,age,income_indice
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,few children,mature,4
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,few children,mature,2
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,few children,young,2
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,бразование,many children,young,4
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,few children,old,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,жилье,few children,mature,4
21521,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,few children,old,2
21522,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,few children,mature,1
21523,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,many children,mature,4


In [36]:
%%HTML
<!DOCTYPE html>
<html>
<head>
<style>
table {
  font-family: arial, sans-serif;
  width: 100%;
}

th{
    text-align: center;
}

</style>
</head>
  
<body>

<h2>Feuille de route</h2>
    <br>
<table>
  <tr>
    <th style="text-align: center; font-size: 1rem; background-color:#666060; color: #f7f5f5;" >Identify and filling missing values </th>
    <th style="text-align: center; font-size: 1rem; background-color:#666060; color: #f7f5f5;">Change data type </th>
    <th style="text-align: center; font-size: 1rem; background-color:#666060; color: #f7f5f5;">Remove duplicates</th>
    <th style="text-align: center; font-size: 1rem; background-color:#666060; color: #f7f5f5;" >Lemmatization</th>
    <th style="text-align: center; font-size: 1rem; background-color:#666060; color: #f7f5f5;">Categorize data</th>
  </tr>
  <tr>
     <td style="text-align: center; font-size: 1rem;"> Drop 'days employed' feature</td>
     <td style="text-align: center; font-size: 1rem;"> Convert 'total_income' feature to int for better visualisation</td>
     <td style="text-align: center; font-size: 1rem;">Removing all duplicate in the data set.<br>Maybe a human error</td>
     <td style="text-align: center; font-size: 1rem;">Lemmatisation of 'purpose' feature to better categorize data</td>
    <td style="text-align: center; font-size: 1rem;"> 'purpose feature' is categorised five big group ('свадьба','недвижимость',<br>'жилье', 'автомобиль', 'бразование')</td>
  </tr>
  <tr>
        <td style="text-align: center; font-size: 1rem;">Deleting row  where children feature is negative</td>
        <td style="text-align: center; font-size: 1rem;"></td>
        <td style="text-align: center; font-size: 1rem;"></td>
        <td style="text-align: center; font-size: 1rem;"></td>
        <td style="text-align: center; font-size: 1rem;">'children' feature can be divided in 2 groups:<br>0 and 1 : few children <br>3 , 4 , 5 : many children</td>
  </tr>
  <tr>
        <td style="text-align: center; font-size: 1rem;">Replace 20 by 2 in 'children feature' </td>
        <td style="text-align: center; font-size: 1rem;"></td>
        <td style="text-align: center; font-size: 1rem;"></td>
        <td style="text-align: center; font-size: 1rem;"></td>
        <td style="text-align: center; font-size: 1rem;">We divide clients age in 3 groups : <br>young (age <=33) <br><br>
                                                                        mature(age >33 and age <=43)<br><br>old((age>43))</td>
  </tr>
  <tr>
        <td style="text-align: center; font-size: 1rem;">Replace '0'  in the 'dob_years feature' by the mean </td>
        <td style="text-align: center; font-size: 1rem;"></td>
        <td style="text-align: center; font-size: 1rem;"></td>
        <td style="text-align: center; font-size: 1rem;"></td>
        <td style="text-align: center; font-size: 1rem;">total_income is divided in 4 groups (1 for total_income  <= 107555)<br><br> 
                                                                    (2 for total_income  > 107555 && total_income  <= 156069)<br><br>
                                                                    (3 for total_income  > 156069 && total_income  <= 195854)<br> <br>
                                                                    (4 for total_income  > 195854 )<br></td>
  </tr>
  <tr>
        <td style="text-align: center; font-size: 1rem;">Lower all value in 'education' feature </td>
        <td style="text-align: center; font-size: 1rem;"></td>
        <td style="text-align: center; font-size: 1rem;"></td>
        <td style="text-align: center; font-size: 1rem;"></td>
        <td style="text-align: center; font-size: 1rem;"></td>
        
  </tr>
  <tr>
        <td style="text-align: center; font-size: 1rem;">Fill the missing value by the mean in 'total_income' feature </td>
         <td style="text-align: center; font-size: 1rem;"></td>
        <td style="text-align: center; font-size: 1rem;"></td>
        <td style="text-align: center; font-size: 1rem;"></td>
        <td style="text-align: center; font-size: 1rem;"></td>
  </tr>
</table>

</body>
</html>

Identify and filling missing values,Change data type,Remove duplicates,Lemmatization,Categorize data
Drop 'days employed' feature,Convert 'total_income' feature to int for better visualisation,Removing all duplicate in the data set.Maybe a human error,Lemmatisation of 'purpose' feature to better categorize data,"'purpose feature' is categorised five big group ('свадьба','недвижимость','жилье', 'автомобиль', 'бразование')"
Deleting row where children feature is negative,,,,"'children' feature can be divided in 2 groups:0 and 1 : few children 3 , 4 , 5 : many children"
Replace 20 by 2 in 'children feature',,,,We divide clients age in 3 groups : young (age <=33) mature(age >33 and age <=43)old((age>43))
Replace '0' in the 'dob_years feature' by the mean,,,,total_income is divided in 4 groups (1 for total_income <= 107555) (2 for total_income > 107555 && total_income <= 156069) (3 for total_income > 156069 && total_income <= 195854) (4 for total_income > 195854 )
Lower all value in 'education' feature,,,,
Fill the missing value by the mean in 'total_income' feature,,,,


## Conclusion Step2:
Using the Describe is useful to group qantitative data .
In this way taking the lower **quartile(Q1)** , **milddle quartile (Q2)** and  **the upper quartile(Q3)**
can help to group data .
for the purpose feature , we used lemmazation.


# Step 3: Answer the questions

+ **1) Is there a relationship between having children and repaying a loan on time?**
> 

In [29]:
# stat for 'few children' feature 
few_children_debt = len(new_clients[(new_clients['children_grouped'] == 'few children') & 
                  (new_clients['debt'] == 1)][['debt','children_grouped']])
# print(few_children_debt)
few_children_total = new_clients[new_clients['children_grouped'] == 'few children'].count()[0]
# print(few_children_total)

math.ceil((few_children_debt / few_children_total) * 100)

8

In [30]:
many_children_debt = len(new_clients[(new_clients['children_grouped'] == 'many children') & 
                  (new_clients['debt'] == 1)][['debt','children_grouped']])
# print(many_children_debt)
many_children_total = new_clients[new_clients['children_grouped'] == 'many children'].count()[0]
math.ceil((many_children_debt / many_children_total) * 100)

10

In [31]:
#using groupby method
print(new_clients.groupby('children_grouped')['debt'].agg('count'))
print('\n')
print(new_clients.groupby('children_grouped')['debt'].agg('sum'))

children_grouped
few children     18899
many children     2508
Name: debt, dtype: int64


children_grouped
few children     1507
many children     233
Name: debt, dtype: int64


In [32]:
def percent(x):
    return "{0:.1%}".format(x)

#using groupby method
children_grouped = new_clients.pivot_table(index = 'children_grouped', values = 'debt', aggfunc=('sum','count','mean'))
children_grouped['mean'] = children_grouped['mean'].apply(percent)
children_grouped = children_grouped.rename(columns={"sum" : "total_debt_client", "count" : "total_client"})
children_grouped

,total_client,mean,total_debt_client
children_grouped,,,
few children,18899,8.0%,1507
many children,2508,9.3%,233


**8%** of of clients who have **few children** have difficulty to  repaying loan.<br>
In countrast **9.3%** of clients who have **many children** have difficulty to repaying loan.<br>

+ **2) Is there a relationship between marital status and loan repayment on time?**
> 

In [33]:
status_familly_grouped =  new_clients.pivot_table(index = 'family_status', values = 'debt', aggfunc=('sum','count','mean'))
status_familly_grouped['mean'] = status_familly_grouped['mean'].apply(percent)
status_familly_grouped.sort_values(by='mean')

,count,mean,sum
family_status,,,
вдовец / вдова,955,6.6%,63
в разводе,1191,7.1%,85
женат / замужем,12310,7.6%,930
гражданский брак,4146,9.4%,388
Не женат / не замужем,2805,9.8%,274


**diffilculty to repay loan acccording the family status in %**
+  вдовец / вдова :  6.6% 
+  в разводе : 7.1%
+  женат / замужем :  7.6%
+  гражданский брак	 :  9.4%
+ Не женат / не замужем : 9.8%


+ **3)Is there a relationship between income level and loan repayment on time?**
> 

In [34]:
income_grouped =  new_clients.pivot_table(index='income_indice', values='debt', aggfunc=('sum','count','mean'))
income_grouped['mean'] = income_grouped['mean'].apply(percent)
income_grouped

,count,mean,sum
income_indice,,,
1,5352,8.0%,427
2,5352,8.7%,467
3,5351,8.7%,463
4,5352,7.2%,383


**diffilculty to repay loan according the income  in %**
+  low total_income  :  8.0% .
+  medium total_income  : 8.7% .
+  high income	 :   8.7% %
+  very hight income : 7.2%%

+ **How do different purposes of a loan affect its repayment on time?**
> 

In [35]:
purpose_grouped =  new_clients.pivot_table(index='purpose_lemma', values='debt', aggfunc=('sum','count','mean'))
purpose_grouped['mean'] = purpose_grouped['mean'].apply(percent)
purpose_grouped.sort_values('mean')

,count,mean,sum
purpose_lemma,,,
жилье,4452,6.9%,308
недвижимость,6335,7.5%,474
свадьба,2322,8.0%,186
бразование,4003,9.2%,370
автомобиль,4295,9.4%,402


**diffilculty to repay loan according the purpose in %**
+  жилье  :  6.9% .
+  недвижимость  : 7.5% .
+  свадьба :   8.0% .
+  бразование : 9.2% .
+  автомобиль : 9.4% .

# Step 4. General conclusion

###### The ideal can be  considering with this feature or parameter as follow:
 - man / woman  
 - few children 
 - divorced / 
 - widower / widow
 - married
 - very hight income
 
 It is possible tor right a template to target the 'ideal client for loan'
